In [ ]:
%load_ext lab_black
import pandas as pd
import altair as alt
import numpy as np
from scipy.optimize import curve_fit
from IPython.display import clear_output
alt.data_transformers.disable_max_rows()

### Load new (Aug 19, 2020) combined dataset 1325

In [ ]:
df = pd.read_csv("1325_sims.csv", index_col=0)

df.rename(
    columns={
        "ID": "code_name",
        "Trial.Scaled": "epoch",
        "Hidden": "hidden_units",
        "PhoHid": "cleanup_units",
        "Pnoise": "p_noise",
        "Epsilon": "learning_rate",
        "Type": "cond",
        "Measure": "measure",
        "Score": "score",
        "Freq": "cond_freq",
        "Cons": "cond_cons",
    },
    inplace=True,
)

df = df.loc[df.measure == "Accuracy"]

### Count model in h-grid

In [ ]:
variates = ["hidden_units", "cleanup_units", "p_noise", "learning_rate"]
settings = df[["code_name"] + variates].pivot_table(index="code_name")
settings["code_name"] = settings.index
settings["learning_rate"] = settings.learning_rate.round(4)

count_settings = settings.pivot_table(
    index=variates, aggfunc="count", values="code_name",
)
count_settings.reset_index(inplace=True)
count_settings.rename(columns={"code_name": "n"}, inplace=True)
count_settings

plot_count = (
    alt.Chart(count_settings)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color="n:O",
        tooltip=variates + ["n"],
    )
    .properties(title="Model counts")
)

plot_count

### Subset to 20 cleanup units and aggregrate within each h-param cell

In [ ]:
sdf = df.loc[
    df.cleanup_units == 20,
]
sdf = sdf.groupby(
    ["epoch", "p_noise", "hidden_units", "learning_rate", "cond"], as_index=False
).mean()
sdf.drop(columns=["code_name", "cleanup_units"], inplace=True)
sdf["code_name"] = sdf.agg(
    lambda x: f'n{x["p_noise"]}_h{x["hidden_units"]}_l{x["learning_rate"]}', axis=1
)

sdf

### Last epoch accuracy in HF_INC

In [ ]:
last_epoch_hfinc = sdf.loc[
    (sdf.epoch == 1.0) & (sdf.cond == "HF_INC"),
]

alt.Chart(last_epoch_hfinc).mark_rect().encode(
    x="p_noise:O",
    y="hidden_units:O",
    column="learning_rate:O",
    color=alt.Color("score", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))),
    tooltip=["score"],
).properties(title="End of training accuracy (HF_INC) in each hyperparameter setting")

### Parse df_wnw

In [ ]:
variates = ["hidden_units", "p_noise", "learning_rate"]

df_wnw = sdf.loc[
    (sdf.cond.isin(["HF_INC", "NW_UN"])),
    variates + ["code_name", "epoch", "cond", "score"],
]

df_wnw = df_wnw.pivot_table(
    index=variates + ["epoch", "code_name"], columns="cond"
).reset_index()

df_wnw.columns = df_wnw.columns = ["".join(c).strip() for c in df_wnw.columns.values]
df_wnw.rename(
    columns={"scoreHF_INC": "word_acc", "scoreNW_UN": "nonword_acc",}, inplace=True,
)

df_wnw["word_advantage"] = df_wnw.word_acc - df_wnw.nonword_acc
df_wnw

### Old dashboard reference

In [ ]:
alt.data_transformers.disable_max_rows()


def main_dashboard(df, df_dev):

    sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

    # df for overview
    df_ov = df[df.epoch == df.epoch.max()]

    # Shared master over-view heatmap
    overview = (
        alt.Chart(df_ov)
        .mark_rect()
        .encode(
            x="p_noise:O",
            y="hidden_units:O",
            row="learning_rate:O",
            color=alt.Color(
                "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
            tooltip=["code_name", "word_acc", "nonword_acc", "word_advantage"],
        )
        .add_selection(sel_run)
        .properties(title="Word accuracy at the end of training")
    )

    wnw_mdf = df.melt(
        id_vars=["code_name", "epoch"],
        value_vars=["word_acc", "nonword_acc"],
        var_name="wnw",
        value_name="acc",
    )

    # Developmental plot
    plot_epoch = (
        alt.Chart(sdf)
        .mark_point()
        .encode(
            y=alt.Y("score:Q", scale=alt.Scale(domain=(0, 1))),
            x="epoch:Q",
            color=alt.Color("cond:N"),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "score"],
        )
        .properties(title="Plot word and nonword accuracy by epoch")
    )

    # Word against Nonword
    wnw_line = (
        alt.Chart(df)
        .mark_line()
        .encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color="learning_rate:Q",
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
        )
    )

    wnw_point = wnw_line.mark_point().encode(
        color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen"))
    )

    diagonal = (
        alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
        .mark_line(color="black")
        .encode(x="x", y="y")
    )

    wnw = diagonal + wnw_line + wnw_point

    wnw_interactive = wnw.add_selection(sel_run).properties(
        title="Word vs. Nonword accuracy at final time step"
    )

    ### Mini heatmap ###

    mini_wadv = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            x="epoch:O",
            color=alt.Color(
                "word_advantage:Q",
                scale=alt.Scale(scheme="redyellowgreen", domain=(-0.3, 0.3)),
            ),
            opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
            tooltip=["word_acc", "nonword_acc", "word_advantage"] + variates,
        )
        .properties(title="Word - Nonword")
    )

    return (overview | (plot_epoch & mini_wadv) | wnw_interactive).resolve_scale(
        y="independent", color="independent", shape="independent"
    )

In [ ]:
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

df_overview = df_wnw.loc[df_wnw.epoch == df_wnw.epoch.max()]

overview = (
    alt.Chart(df_overview)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        color=alt.Color(
            "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["code_name", "word_acc", "nonword_acc", "word_advantage"],
    )
    .add_selection(sel_run)
    .properties(title="Word accuracy at the end of training")
)

overview

df_wnw.sort_values(by=["code_name", "epoch"], inplace=True)

# Word against Nonword
wnw_line = (
    alt.Chart(df_wnw)
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color="code_name:N",
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)

wnw = diagonal + wnw_line

wnw_interactive = wnw.add_selection(sel_run).properties(
    title="Word vs. Nonword accuracy at final time step"
)

overview | wnw_interactive

# Replicating HS99 Fig 27

### Effect of reducing learning rate

In [ ]:
lr_df = df_wnw.loc[(df_wnw.hidden_units == 100) & (df_wnw.p_noise == 0.0)]


wnw_line = (
    alt.Chart(lr_df)
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color("learning_rate", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)

diagonal + wnw_line

### Effect of all learning rate

In [ ]:
from altair.expr import datum

wnw_point = (
    alt.Chart(df_wnw)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color("learning_rate:O", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)


base = wnw_point + wnw_point.transform_loess(
    "word_acc", "nonword_acc", groupby=["learning_rate"], bandwidth=0.5
).mark_line(size=4)

gplot = alt.vconcat()


for noise in np.sort(df_wnw.p_noise.unique())[::-1]:
    plot = alt.hconcat()
    for h in df_wnw.hidden_units.unique():
        plot |= base.transform_filter(
            (datum.hidden_units == h) & (datum.p_noise == noise)
        ).properties(title=f"hidden units = {h}, Pnoise = {noise}")
    gplot &= plot

gplot.save("effect_lr.html")

### Effect of reducing hidden units (all levels)

In [ ]:
hu_df = df_wnw.loc[(df_wnw.p_noise == 0.0) & (df_wnw.learning_rate == 0.01)]


wnw_line = (
    alt.Chart(hu_df)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color("hidden_units", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)

wnw_line + wnw_line.transform_loess(
    "word_acc", "nonword_acc", groupby=["hidden_units"], bandwidth=0.5
).mark_line(size=4)

### Effect of reducing hidden units (<=100 HU)

In [ ]:
hu_df = df_wnw.loc[
    (df_wnw.hidden_units <= 100)
    & (df_wnw.p_noise == 0.0)
    & (df_wnw.learning_rate == 0.01)
]


wnw_line = (
    alt.Chart(hu_df)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color("hidden_units", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)


wnw_line + wnw_line.transform_loess(
    "word_acc", "nonword_acc", groupby=["hidden_units"], bandwidth=0.5
).mark_line(size=4)

### All hidden unit effects

In [ ]:
from altair.expr import datum

wnw_point = (
    alt.Chart(df_wnw)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color("hidden_units:O", scale=alt.Scale(scheme="redyellowgreen")),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)


base = wnw_point + wnw_point.transform_loess(
    "word_acc", "nonword_acc", groupby=["hidden_units"], bandwidth=0.5
).mark_line(size=4)

gplot = alt.vconcat()


for noise in np.sort(df_wnw.p_noise.unique())[::-1]:
    plot = alt.hconcat()
    for lr in df_wnw.learning_rate.unique():
        plot |= base.transform_filter(
            (datum.learning_rate == lr) & (datum.p_noise == noise)
        ).properties(title=f"leanrning rate = {lr}, Pnoise = {noise}")
    gplot &= plot

gplot.save("effect_hidden.html")

### Invasive P lesion in HS99 at development space

In [ ]:
hs99 = pd.read_csv("hs99_severe.csv")
hs99_w = (
    alt.Chart(hs99)
    .mark_line(color="blue")
    .encode(
        x=alt.X("epoch", scale=alt.Scale(domain=(0, 10))),
        y=alt.Y("w_acc", scale=alt.Scale(domain=(0, 100))),
    )
)

hs99_nw = (
    alt.Chart(hs99)
    .mark_line(color="red")
    .encode(
        x=alt.X("epoch", scale=alt.Scale(domain=(0, 10))),
        y=alt.Y("nw_acc", scale=alt.Scale(domain=(0, 100))),
    )
)

hs99_w + hs99_nw

### Invasive P lesion in HS99 at performance space

In [ ]:
point = (
    alt.Chart(hs99)
    .mark_point()
    .encode(
        x=alt.X("w_acc", scale=alt.Scale(domain=(0, 100))),
        y=alt.Y("nw_acc", scale=alt.Scale(domain=(0, 100))),
        color="epoch:Q",
    )
)

line = (
    alt.Chart(hs99)
    .mark_line()
    .encode(
        x=alt.X("w_acc", scale=alt.Scale(domain=(0, 100))),
        y=alt.Y("nw_acc", scale=alt.Scale(domain=(0, 100))),
        order="epoch",
    )
)

point + line

### P-noise effect

In [ ]:
noise_df = df_wnw.loc[(df_wnw.hidden_units == 100) & (df_wnw.learning_rate == 0.01)]
noise_df.columns

wnw_line = (
    alt.Chart(noise_df)
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color(
            "p_noise", scale=alt.Scale(scheme="redyellowgreen", reverse=True)
        ),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)

diagonal + wnw_line

#### Smoothing

In [ ]:
noise_point = wnw_line.mark_point()

noise_point + noise_point.transform_loess(
    "word_acc", "nonword_acc", groupby=["p_noise"], bandwidth=0.5
).mark_line(size=4)

### P_noise full range effect at hidden = 75

In [ ]:
noise_df = df_wnw.loc[(df_wnw.hidden_units == 75) & (df_wnw.learning_rate == 0.01)]
noise_df.columns

wnw_line = (
    alt.Chart(noise_df)
    .mark_line()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color(
            "p_noise", scale=alt.Scale(scheme="redyellowgreen", reverse=True)
        ),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)

diagonal = (
    alt.Chart(pd.DataFrame({"x": [0, 1], "y": [0, 1]}))
    .mark_line(color="black")
    .encode(x="x", y="y")
)

diagonal + wnw_line

In [ ]:
noise_point = wnw_line.mark_point()

noise_point + noise_point.transform_loess(
    "word_acc", "nonword_acc", groupby=["p_noise"], bandwidth=0.5
).mark_line(size=4)

### All p-noise effect

In [ ]:
from altair.expr import datum

noise_df = df_wnw
noise_df.columns


wnw_point = (
    alt.Chart(noise_df)
    .mark_point()
    .encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        order=["epoch"],
        color=alt.Color(
            "p_noise", scale=alt.Scale(scheme="redyellowgreen", reverse=True)
        ),
        tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
    )
)


base = wnw_point + wnw_point.transform_loess(
    "word_acc", "nonword_acc", groupby=["p_noise"], bandwidth=0.5
).mark_line(size=4)

gplot = alt.vconcat()


for h in df_wnw.hidden_units.unique():
    plot = alt.hconcat()
    for lr in df_wnw.learning_rate.unique():
        plot |= base.transform_filter(
            (datum.learning_rate == lr) & (datum.hidden_units == h)
        ).properties(title=f"leanrning rate = {lr}, hidden units = {h}")
    gplot &= plot

gplot.save("effect_p_noise.html")

# Part II: will there be a HF group

In [ ]:
df = pd.read_csv("1325_sims.csv", index_col=0)

df.rename(
    columns={
        "ID": "code_name",
        "Trial.Scaled": "epoch",
        "Hidden": "hidden_units",
        "PhoHid": "cleanup_units",
        "Pnoise": "p_noise",
        "Epsilon": "learning_rate",
        "Type": "cond",
        "Measure": "measure",
        "Score": "score",
        "Freq": "cond_freq",
        "Cons": "cond_cons",
    },
    inplace=True,
)

df = df.loc[df.measure == "Accuracy"]

In [ ]:
df_part2 = df.loc[
    df.cleanup_units == 20,
]
df_part2.columns
df_part2.drop(columns=["cleanup_units", "measure"], inplace=True)

#### Model level performance grouping 0-25, 25-75, 75-100 percentile

In [ ]:
gacc = df_part2.groupby("code_name", as_index=False).mean()
gacc = gacc[["code_name", "score"]]
gacc["rank_pc"] = gacc.score.rank(pct=True)
gacc["group"] = gacc.rank_pc.map(
    lambda x: "High" if x > 0.75 else ("Mid" if x > 0.25 else "Low")
)

#### Create a new df for grouping

In [ ]:
df_group = (
    df_part2.merge(gacc[["code_name", "group"]], how="left")
    .groupby(["group", "epoch", "cond"], as_index=False)
    .mean()
)

In [ ]:
alt.Chart(df_group).mark_line().encode(
    x="epoch:Q", y="score:Q", color="cond:N", column="group:N"
)

In [ ]:
df_wnw_group = df_group.pivot_table(
    index=["group", "epoch"], columns="cond", values="score"
).reset_index()

alt.Chart(df_wnw_group).mark_line().encode(x="HF_INC", y="NW_UN", color="group")

In [ ]:
df_group_all = df_part2.merge(gacc[["code_name", "group"]], how="left")

df_wnw_group_all = df_group_all.pivot_table(
    index=["group", "epoch", "code_name"], columns="cond", values="score"
).reset_index()

alt.Chart(
    df_wnw_group_all.loc[df_wnw_group_all.group.isin(["Mid", "High"])],
).mark_boxplot().encode(x="HF_INC", y="NW_UN", color="group",).properties(
    width=800
)

# Refit growth model with robust von bertaleffy 

In [ ]:
def vonb(x, max_acc, k, x0):
    """ von Bertalanffy (1938)
    Assume that the rate of growth of an organism declines with size 
    so that the rate of change in length, l,  may be described by:
    dl/dt = K (L_inf - l) or under our context: dy/dx = k (max_acc - y)
    max_acc: Maximum accuracy / upper asymtote
    k: growth rate
    x0: x value where model start to learn
    """
    return max_acc * (1 - np.exp(-k * (x - x0)))


def get_df(df, code_name, cond, remove_zero=False):
    """
    Convienient function for subsetting data
    """
    data = df.loc[
        (df.code_name == code_name)
        & (df.cond == cond)
        & (df.measure == "Accuracy"),  # Early points are too volatile
        ["epoch", "score"],
    ].reset_index(drop=True)

    if remove_zero:
        data = data.loc[
            data.score > 0,
        ]

    return data


class growth_model:
    def __init__(self, df, growth_fx, bounds, name, robust=False, f_scale=0.02):
        """ Fit growth models
        df: datafile with score (y) and epoch (x)
        growth_fx: growth function to fit
        bounds: model constrain of parameters
        name: model name
        robust: whether to use robust method
        f_scale (only or robust = True): soft residual cutoff for outlier, 
            used in scipy.optimize.least_squares()
        See https://scipy-cookbook.readthedocs.io/items/robust_regression.html
        for more details
        """
        self.df = df
        self.growth_fx = growth_fx
        self.bounds = (-np.inf, np.inf) if bounds == None else bounds
        self.__name__ = name
        self.f_scale = f_scale

        # Create plotting dataset
        self.df["set"] = "actual"
        self.actual = self.df.score
        self.n = len(self.df)

        # Fit model
        self.fit_robust() if robust else self.fit()

        # Predicts
        self.pred = self.growth_fx(self.df.epoch, *self.params)
        self.res = np.sum(np.square(self.pred - self.actual))
        self.adj_res = self.res * 19 / self.n

        model_df = pd.DataFrame(
            {"epoch": self.df.epoch, "score": self.pred, "set": "predict",}
        )
        self.df = pd.concat([self.df, model_df], ignore_index=True)

    def fit(self):

        self.params, _ = curve_fit(
            self.growth_fx, self.df.epoch, self.df.score, bounds=self.bounds
        )

    def fit_robust(self):
        """ Fitting the selected curve with robust method
        """
        self.params, _ = curve_fit(
            self.growth_fx,
            self.df.epoch,
            self.df.score,
            bounds=self.bounds,
            maxfev=10000,
            method="trf",  # Relatively robust method
            loss="soft_l1",  # More robust to outlier
            f_scale=self.f_scale,  # Soft boundary for outlier residual, based on specific data set,
        )

    def plot(self):
        return (
            alt.Chart(self.df)
            .mark_line(point=True)
            .encode(
                y=alt.Y("score", scale=alt.Scale(domain=(0, 1))),
                x=alt.X("epoch", scale=alt.Scale(domain=(0, 1))),
                color="set",
            )
            .properties(
                title=[
                    f"Model: {self.__name__}",
                    f"Parameters: {self.params.round(3)}",
                    f"Residual (SSE): {self.res:.3f}",
                    f"Data point adjusted residual {self.adj_res:.3f}",
                ]
            )
        )

In [ ]:
sel_df = df.loc[
    df.code_name == df.code_name.unique()[1292],
]

sel_df

In [ ]:
for c in df.cond.unique():
    print(c)
    tmp = get_df(df, 163115474, c, remove_zero=True)
    print(tmp)

# Fit growth model in each run x condition

In [ ]:
i = 0
n = len(df.code_name.unique())

results = pd.DataFrame()

for m in df.code_name.unique():
    clear_output(wait=True)
    i += 1
    print("Processing model {} of {}".format(i, n))
    for c in df.cond.unique():
        m1 = growth_model(
            get_df(df, m, c, remove_zero=True),
            vonb,
            (0, [1, np.inf, 1]),
            "rEQ1",
            True,
            0.01,
        )
        m1_result = pd.DataFrame(
            {
                "model": "EQ1",
                "adj_residual": m1.adj_res,
                "max_acc": m1.params[0],
                "k": m1.params[1],
                "x0": m1.params[2],
            },
            index=[0],
        )
        m1_result["code_name"] = m
        m1_result["cond"] = c

        results = pd.concat([results, m1_result])

In [ ]:
model_spec = df.groupby("code_name").mean().reset_index()
model_spec.drop(columns=["epoch", "score"], inplace=True)

robust_eq1_results = robust_eq1_results.reset_index()
robust_eq1_results = robust_eq1_results.merge(model_spec, on="code_name")
robust_eq1_results.to_csv("growth_1325.csv")
print(f"EQ 1 mean residual = {robust_eq1_results.adj_residual.mean():.4f}")